In [24]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [25]:
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json
import pickle
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers import SGD
from keras.models import load_model
import random
import warnings
warnings.filterwarnings('ignore')

In [26]:
nltk.download('punkt')

nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [27]:
!unzip /usr/share/nltk_data/corpora/wordnet.zip -d /usr/share/nltk_data/corpora/

unzip:  cannot find or open /usr/share/nltk_data/corpora/wordnet.zip, /usr/share/nltk_data/corpora/wordnet.zip.zip or /usr/share/nltk_data/corpora/wordnet.zip.ZIP.


In [28]:
words=[]
classes = []
documents = []
ignore_words = ['?', '!']
data = open('drive/MyDrive/Chatbot/Klapa_Chatbot_Dataset.json').read()
intents = json.loads(data)

# Check keys and firs five data in key value

In [29]:
intents.keys()

dict_keys(['intents'])

In [30]:
intents['intents'][:5]

[{'tag': 'sapaan',
  'patterns': ['Hai',
   'Selamat pagi',
   'Selamat siang',
   'Selamat malam',
   'Apa kabar?',
   'Halo',
   'Hei',
   'halo kak'],
  'responses': ['Hai! Selamat datang di toko online kami.',
   'Selamat datang! Apa yang bisa kami bantu?',
   'Halo! Ada yang bisa kami bantu?',
   'Baik, ada yang bisa kami bantu?',
   'Hei! Selamat berbelanja.'],
  'context_set': ''},
 {'tag': 'bertanya',
  'patterns': ['saya ingin bertanya',
   'saya bertanya',
   'bagaimana ini?',
   'saya butuh bantuan'],
  'responses': ['Saya siap menjawab berbagai pertanyaan untuk membantu anda.'],
  'context_set': ''},
 {'tag': 'perpisahan',
  'patterns': ['Sampai jumpa',
   'Selamat tinggal',
   'Terima kasih',
   'Sampai jumpa lagi',
   'Goodbye',
   'Bye',
   'Sampai bertemu lagi',
   'Terima kasih sudah menjawab.',
   'Selamat tinggal, semoga harimu menyenangkan',
   'Bye bye',
   'Sampai jumpa nanti'],
  'responses': ['Sampai jumpa! Jangan ragu untuk kembali.',
   'Terima kasih sudah ber

#  Preprocess data
Before creating a machine learning or deep learning model when working with text data, there are a number of preprocessing steps that must be taken. We need to preprocess the data using a variety of processes depending on the needs.

The simplest and initial action you can do with text data is to tokenize it. The method of tokenizing involves dividing the entire text into smaller units, such as words.

Using the nltk.word_tokenize() method, we tokenize the phrase in this case and attach each word from the words list after iterating over the patterns. For our tags, we also make a list of classes.

In [31]:
for intent in intents['intents']:
    for pattern in intent['patterns']:
        #tokenize each word
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        #add documents in the corpus
        documents.append((w, intent['tag']))
        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

We will now lemmatize each word and eliminate any repetitions from the list. Lemmatizing entails taking a word and changing it into a lemma. After that, a pickle file is made to house the Python objects that will be used for prediction.

In [32]:
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))
# sort classes
classes = sorted(list(set(classes)))
# documents = combination between patterns and intents
print (len(documents), "documents")
# classes = intents
print (len(classes), "classes", classes)
# words = all words, vocabulary
print (len(words), "unique lemmatized words", words)
#If you save this two list comment out this code and run again


194 documents
26 classes ['aplikasi', 'bahasa kotor', 'bertanya', 'biayapengiriman', 'cancel pemesanan', 'cicilan', 'cod', 'hapus keranjang', 'harga produk', 'kategori produk', 'kebijakan keranjang', 'keranjang', 'nama', 'opsikirim', 'pembayaran', 'pemesanan', 'pencipta', 'pengembalian', 'pengiriman', 'perpisahan', 'produk', 'promo', 'random', 'sapaan', 'tambah keranjang', 'total harga']
219 unique lemmatized words [',', '.', 'ada', 'ajak', 'akan', 'akankah', 'aku', 'anda', 'anjing', 'apa', 'apakah', 'aplikasi', 'as', 'asshole', 'atau', 'aturan', 'bagaimana', 'bajingan', 'bangsat', 'bantuan', 'barang', 'batal', 'bayar', 'belanja', 'beli', 'berapa', 'berikan', 'berlaku', 'berlangsung', 'bertanya', 'bertemu', 'biaya', 'bicara', 'bisa', 'bisakah', 'bitch', 'blok', 'bodoh', 'butuh', 'bye', 'cancel', 'cara', 'cash', 'cek', 'checkout', 'cicil', 'cicilan', 'cinta', 'cod', 'cogil', 'creator', 'dalam', 'dana', 'dari', 'delivery', 'dengan', 'denganku', 'developer', 'developermu', 'di', 'dibuat',

In [33]:
pickle.dump(words,open('words.pkl','wb'))
pickle.dump(classes,open('classes.pkl','wb'))

# Create training and testing data
Now that the training data has been created, the input and output will be included. The pattern will serve as our input, and the class to which the pattern belongs will serve as our output. However, since the machine cannot interpret language, we must convert it to numbers.

In [34]:
# create our training data
training = []
# create an empty array for our output
output_empty = [0] * len(classes)
# training set, bag of words for each sentence
for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # lemmatize each word - create base word, in attempt to represent related words
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    # create our bag of words array with 1, if word match found in current pattern
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)
    # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    training.append([bag, output_row])
# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training)
# create train and test lists. X - patterns, Y - intents
train_x = list(training[:,0])
train_y = list(training[:,1])
print("Training data created")

Training data created


# Build the model

In [35]:
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))
# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
optimizer = SGD(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
#fitting and saving the model
hist = model.fit(np.array(train_x), np.array(train_y), epochs=500, batch_size=5, verbose=1)


Epoch 1/500
39/39 [==============================] - 1s 2ms/step - loss: 3.2573 - accuracy: 0.0361
Epoch 2/500
39/39 [==============================] - 0s 2ms/step - loss: 3.2331 - accuracy: 0.0670
Epoch 3/500
39/39 [==============================] - 0s 2ms/step - loss: 3.2027 - accuracy: 0.0773
Epoch 4/500
39/39 [==============================] - 0s 2ms/step - loss: 3.1734 - accuracy: 0.1031
Epoch 5/500
39/39 [==============================] - 0s 2ms/step - loss: 3.1582 - accuracy: 0.0979
Epoch 6/500
39/39 [==============================] - 0s 2ms/step - loss: 3.1538 - accuracy: 0.1340
Epoch 7/500
39/39 [==============================] - 0s 2ms/step - loss: 3.1397 - accuracy: 0.0773
Epoch 8/500
39/39 [==============================] - 0s 2ms/step - loss: 3.1143 - accuracy: 0.1392
Epoch 9/500
39/39 [==============================] - 0s 2ms/step - loss: 3.1318 - accuracy: 0.1340
Epoch 10/500
39/39 [==============================] - 0s 2ms/step - loss: 3.1017 - accuracy: 0.1289
Epoch 11/

> If You save the model then comment out the cell bellow and run this

In [36]:
model.save('chatbot_model.h5', hist)
print("model created")

model created


In [37]:
model = load_model('chatbot_model.h5')

In [38]:
intents = json.loads(open('drive/MyDrive/Chatbot/Klapa_Chatbot_Dataset.json').read())

words = pickle.load(open('words.pkl','rb'))

classes = pickle.load(open('classes.pkl','rb'))

In [39]:
def clean_up_sentence(sentence):
  # tokenize the pattern - split words into array
  sentence_words = nltk.word_tokenize(sentence)

  # stem each word - create short form for word
  sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]

  return sentence_words

In [40]:
def bow(sentence, words, show_details=True):
  # tokenize the pattern
  sentence_words = clean_up_sentence(sentence)

  # bag of words - matrix of N words, vocabulary matrix
  bag = [0]*len(words)


  for s in sentence_words:
      for i,w in enumerate(words):
          if w == s:
            # assign 1 if current word is in the vocabulary position
            bag[i] = 1
            if show_details:
              print ("found in bag: %s" % w)


  return(np.array(bag))

In [41]:
def predict_class(sentence, model):
  # filter out predictions below a threshold
  p = bow(sentence, words,show_details=False)

  res = model.predict(np.array([p]))[0]

  ERROR_THRESHOLD = 0.25

  results = [[i,r] for i,r in enumerate(res) if r>ERROR_THRESHOLD]

  # sort by strength of probability
  results.sort(key=lambda x: x[1], reverse=True)


  return_list = []

  for r in results:
      return_list.append({"intent": classes[r[0]], "probability": str(r[1])})

  return return_list

In [42]:
def getResponse(ints, intents_json):

  tag = ints[0]['intent']

  list_of_intents = intents_json['intents']
  #print(list_of_intents)

  for i in list_of_intents:
    if(i['tag']== tag):
      result = random.choice(i['responses'])
      break
  return result

In [43]:
def chatbot_response(text):
  ints = predict_class(text, model)
  res = getResponse(ints, intents)

  return res

In [ ]:
start = True

while start:
    query = input('Enter Message: ')

    if query.lower() in ['quit', 'exit', 'bye']:
        start = False
        continue

    try:
        res = chatbot_response(query)
        print(res)
    except Exception as e:
        print(f'An error occurred: {e}. You may need to rephrase your question.')

1/1 [==============================] - 0s 58ms/step
Hei! Selamat berbelanja.
1/1 [==============================] - 0s 28ms/step
Saya siap menjawab berbagai pertanyaan untuk membantu anda.
1/1 [==============================] - 0s 21ms/step
Tolong gunakan bahasa yang baik.
1/1 [==============================] - 0s 19ms/step
Tolong gunakan bahasa yang baik.
1/1 [==============================] - 0s 31ms/step
Saya siap menjawab berbagai pertanyaan untuk membantu anda.
1/1 [==============================] - 0s 21ms/step
Saya hanya menjawab pertanyaan yang sopan.
1/1 [==============================] - 0s 21ms/step
Untuk menambahkan produk ke keranjang, kunjungi halaman produk, pilih produk yang diinginkan, dan klik tombol 'Tambahkan ke Keranjang'.
1/1 [==============================] - 0s 28ms/step
Tidak, kami hanya menyediakan pembayaran melalui transfer bank, dan metode pembayaran online lainnya.
